In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import pandas as pd
import multiprocessing as mp
import sys
import threading
import re
import numpy as np

In [2]:
def get_speeches(url1, url2, letter, return_dict):
    if letter == 'c':
        r = requests.get(url2.format(letter))
        soup = BeautifulSoup(r.text, 'html.parser')
        content_list = soup.find('div', attrs={'class':'site-content'})
    else:
        r = requests.get(url1.format(letter))
        soup = BeautifulSoup(r.text, 'html.parser')
        content_list = soup.find('div', attrs={'class':'entry-content'})
    speeches = []
    for result in tqdm_notebook(content_list.findAll('a', href=True)):
        if result.text != 'Speeches Index':
            try:
                speech_url = result.get('href')
                speech_r = requests.get(speech_url)
                speech_soup = BeautifulSoup(speech_r.text, 'html.parser')
                speech_text = speech_soup.find('div', attrs={'class':'entry-content'}).text
                speeches.append([result.text,speech_text])
            except:
                try:
                    speeches.append([result.text,'unable to scrape'])
                except:
                    pass
    return_dict[letter]=speeches

    
def request_thread(url1, url2, letters):
    manager = mp.Manager()
    return_dict = manager.dict()
    threads = []
    for letter in tqdm_notebook(letters):
        thread = threading.Thread(name=letter, 
                                  target=get_speeches, 
                                  args=(url1, url2, letter, return_dict))
        thread.start()
        threads.append(thread)

    for t in threads:
        t.join()

    return return_dict

In [3]:
from string import ascii_lowercase

In [4]:
url1 = "http://www.ukpol.co.uk/speeches/speeches-{}/"
url2 = "http://www.ukpol.co.uk/speeches/{}/"

In [5]:
my_letters = ascii_lowercase
for letter in my_letters:
    if letter == 'x':
        my_letters = my_letters.replace(letter,'')

In [6]:
speeches_dict = request_thread(url1, url2, my_letters)

In [7]:
speeches = pd.DataFrame([x for letter in speeches_dict.values() for x in letter])

In [8]:
speeches.columns = ['title','speech']

In [9]:
speeches.head()

,title,speech
0,"Isherwood, Mark – 2006 Speech on Fuel Poverty",\nBelow is the text of the speech made by Mark...
1,"Zahawi, Nadhim – 2018 Speech at National Learn...",\n\nBelow is the text of the speech made by Na...
2,"Zahawi, Nadhim – 2018 Statement on Childcare",\n\nBelow is the text of the statement made by...
3,"Zambellas, George – 2016 Speech on the Royal Navy",\n\nBelow is the text of the speech made by Si...
4,"Zeichner, Daniel – 2015 Maiden Speech in the H...",\nBelow is the text of the maiden speech made ...


In [10]:
speeches.shape

(3089, 2)

Got 12 more speeches, not worth cleaning them all. Might rescrape in the next few weeks.

In [12]:
letters = speeches_dict.keys()
speech_count = [len(x) for x in speeches_dict.values()]

In [13]:
count_check = pd.DataFrame({'letter':letters, 'speech_count':speech_count})

In [15]:
count_check.sort_values('speech_count')

,letter,speech_count
0,i,1
1,z,6
2,y,7
3,u,7
4,n,12
5,e,23
6,v,35
7,k,49
8,r,55
9,q,56
